In [1]:
# Do standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Do additional imports
import logomaker
import glob
import re
import os

In [2]:
# Set mapping from strain numbers to mutations
mutant_to_strain_dict = {'m1':'Y486Q', 'm20':'R478K', 'm21':'FY-AA', 
                         'm22':'FY-IQ', 'm23':'FY-YF', 'm3':'F485I', 
                         'm5':'N489A', 'm6':'N489W', 'm9':'R478A', 
                         'wt':'WT', 
                         'm2':'F485I'} # I think m2 is a typo and corresponds to m3

In [3]:
# Set imput directoreis
dir_IM_raw = 'models_IM_multirun'
dir_IM = 'models_IM'
dir_ER = 'models_ER'

# Create set of data columns
L = 50
data_cols = ['%02d.%s'%(n,c) for n in range(L) for c in 'ACGT']

## Average IM matrices

In [4]:
# Get list of raw matrices
mat_names = glob.glob(f'{dir_IM_raw}/*.txt')
print('There are %d raw IM files to process.'%len(mat_names))

# Create names for averaged matrices
avg_names = list(set([s[:-7]+'.txt' for s in mat_names]))
print('Saving %d averaged IM files.'%len(avg_names))

# Clear directory of averaged matrices
files = glob.glob(f'{dir_IM}/*.txt')
for f in files:
    os.remove(f)

# Create each averaged matrix one by one
for avg_name in avg_names:
    
    # Get list of relevant raw matrix names
    these_mat_names = glob.glob(avg_name[:-4]+'_v*.txt')
    
    # Load data frames and average (this code is a little strange looking, but it works)
    dfs = []
    for v, name in enumerate(these_mat_names):
        df = pd.read_csv(name, delim_whitespace=True)
        dfs.append(df)
    df_cat = pd.concat(dfs, ignore_index=True)
    df_cat.dropna(axis=0, inplace=True)
    df_avg = df_cat.groupby(by='pos').mean()
    
    # Make sure dataframe is centered and normalized
    df_avg.loc[:,:] = df_avg.values - df_avg.values.mean(axis=1)[:,np.newaxis]
    df_avg.loc[:,:] = df_avg.values / np.sqrt(np.sum(df_avg.values.ravel()**2))
    
    # save the dataframe
    avg_file_name = f'{dir_IM}/'+avg_name.split('/')[-1][:-4]+'.txt'
    print('.',end='')
    df_avg.to_csv(avg_file_name, sep='\t')
    
print('\nDone.')

There are 300 raw IM files to process.
Saving 60 averaged IM files.
............................................................
Done.


In [5]:
# Check to make sure there are the same number of IM_avg and ER matrices
num_IM_files = len(glob.glob(f'{dir_IM}/*.txt'))
num_ER_files = len(glob.glob(f'{dir_ER}/*.txt'))
print('num_IM_files:', num_IM_files)
print('num_ER_files:', num_ER_files)
if num_IM_files == num_ER_files:
    print('Same number of averaged IM files and ER files')
else:
    print('WARNING! Different number of averaged IM and ER files')

num_IM_files: 60
num_ER_files: 60
Same number of averaged IM files and ER files


In [6]:
# Load ER models into dataframe
LM = 'ER'
file_glob = f'{dir_ER}/model.ER.*.txt'
file_pattern = '.*/model.ER.(.*)_v1.txt'
name_pattern = '(exp[0-9]+)_(ars[0-9]+)_([a-z0-9\-_]+)'
df_file = 'data_ER.txt'

# Load models indictionary format
model_files = glob.glob(file_glob)

# Filter out 'all' from files
model_files = [f for f in model_files if not 'all' in f]
model_files = [f for f in model_files if not '-' in f]
model_df_dict = {}
print('Loading %d models into model_df_dict...'%len(model_files))
for model_file in model_files:
    model_name = re.match(file_pattern, model_file).group(1)
    model_df = pd.read_csv(model_file, sep='\t', index_col=0)
    model_df_dict[model_name] = model_df

# Save model info in dataframe
model_df_list = list(model_df_dict.values())
N = len(model_df_dict)
P = len(model_df_list[0].values.ravel())
print('N = %d, P = %d'%(N,P))    
assert(P==4*L)

# Fill in data_df, which will be used for PCA
data_df = pd.DataFrame(data=np.zeros([N,P]), index=model_df_dict.keys(), columns=data_cols)
for name, row in data_df.iterrows():
    tmp_df = model_df_dict[name].copy()
    vec = tmp_df.values.ravel()
    vec /= np.sqrt(np.sum(vec**2))
    row[:] = vec
data_cols = list(data_df.columns).copy()

matches = [re.match(name_pattern,name) for name in data_df.index]

data_df['exp'] = [m.group(1) for m in matches]
data_df['ars'] = [m.group(2) for m in matches]
data_df['strain'] = [m.group(3) for m in matches]
short_strains = [re.match('([^c]+).*',strain).group(1) for strain in data_df['strain']]
data_df['mut'] = [mutant_to_strain_dict[s] for s in short_strains]
data_df.index.name='name'
data_df.reset_index(inplace=True)
info_cols = ['name','ars','mut','strain','exp'] 

# Rearrange cols and sort rows in data_df
data_df = data_df[info_cols + data_cols]
data_df.sort_values(by=['ars','mut','exp'], inplace=True)
#data_df.reset_index(inplace=True, drop=True)
data_df.set_index('name', inplace=True, drop=True)

# Make sure names are unique
names = data_df.index
print(len(set(names))==len(names))

data_df.to_csv(df_file, sep='\t')
print(f'Data written to {df_file}.')
data_df.tail()

Loading 60 models into model_df_dict...
N = 60, P = 200
True
Data written to data_ER.txt.


,ars,mut,strain,exp,00.A,00.C,00.G,00.T,01.A,01.C,...,47.G,47.T,48.A,48.C,48.G,48.T,49.A,49.C,49.G,49.T
name,,,,,,,,,,,,,,,,,,,,,
exp9_ars416_wtc6,ars416,WT,wtc6,exp9,0.000431,-0.011029,-0.013799,0.024397,0.017172,-0.006898,...,-0.070398,0.074860,-0.012950,0.015406,-0.008831,0.006375,0.016932,-0.004225,0.014264,-0.026971
exp4_ars416_m1,ars416,Y486Q,m1,exp4,0.008098,-0.004522,0.009692,-0.013268,0.019458,-0.007170,...,-0.055464,0.084113,0.000417,0.009325,-0.014738,0.004996,0.037443,-0.001836,0.022535,-0.058143
exp7_ars416_m1,ars416,Y486Q,m1,exp7,0.004696,0.003284,0.000006,-0.007986,0.000589,-0.004122,...,-0.071807,0.100187,-0.009799,0.023625,-0.012448,-0.001378,0.033399,-0.012317,0.029282,-0.050364
exp9_ars416_m1c1,ars416,Y486Q,m1c1,exp9,0.001505,0.001767,0.004366,-0.007638,0.004164,-0.000890,...,-0.124055,0.109687,-0.010071,0.021173,-0.010648,-0.000453,0.033394,-0.011913,0.025440,-0.046921
exp9_ars416_m1c2,ars416,Y486Q,m1c2,exp9,0.003425,-0.004637,0.004153,-0.002940,0.003544,-0.002033,...,-0.074113,0.103688,-0.015597,0.020873,-0.004093,-0.001183,0.032072,-0.011380,0.023952,-0.044643


In [7]:
# Load IM models into dataframe
LM = 'IM'
file_glob = f'{dir_IM}/model.mi.*.txt'
file_pattern = '.*/model.mi.(.*).txt'
name_pattern = '(exp[0-9]+)_(ars[0-9]+)_([a-z0-9\-_]+)'
df_file = 'data_IM.txt'

# Load models indictionary format
model_files = glob.glob(file_glob)

# Filter out 'all' from files
model_files = [f for f in model_files if not 'all' in f]
model_files = [f for f in model_files if not '-' in f]
model_df_dict = {}
print('Loading %d models into model_df_dict...'%len(model_files))
for model_file in model_files:
    model_name = re.match(file_pattern, model_file).group(1)
    model_df = pd.read_csv(model_file, sep='\t', index_col=0)
    model_df_dict[model_name] = model_df

# Save model info in dataframe
model_df_list = list(model_df_dict.values())
N = len(model_df_dict)
P = len(model_df_list[0].values.ravel())
print('N = %d, P = %d'%(N,P))    

# Fill in data_df, which will be used for PCA
data_df = pd.DataFrame(data=np.zeros([N,P]), index=model_df_dict.keys(), columns=data_cols)
for name, row in data_df.iterrows():
    tmp_df = model_df_dict[name].copy()
    vec = tmp_df.values.ravel()
    vec /= np.sqrt(np.sum(vec**2))
    row[:] = vec
data_cols = list(data_df.columns).copy()

matches = [re.match(name_pattern,name) for name in data_df.index]

data_df['exp'] = [m.group(1) for m in matches]
data_df['ars'] = [m.group(2) for m in matches]
data_df['strain'] = [m.group(3) for m in matches]
short_strains = [re.match('([^c]+).*',strain).group(1) for strain in data_df['strain']]
data_df['mut'] = [mutant_to_strain_dict[s] for s in short_strains]
data_df.index.name='name'
data_df.reset_index(inplace=True)
info_cols = ['name','ars','mut','strain','exp'] 

# Rearrange cols and sort rows in data_df
data_df = data_df[info_cols + data_cols]
data_df.sort_values(by=['ars','mut','exp'], inplace=True)
#data_df.reset_index(inplace=True, drop=True)
data_df.set_index('name', inplace=True, drop=True)

# Make sure names are unique
names = data_df.index
print(len(set(names))==len(names))

data_df.to_csv(df_file, sep='\t')
print(f'Data written to {df_file}.')
data_df.tail()

Loading 60 models into model_df_dict...
N = 60, P = 200
True
Data written to data_IM.txt.


,ars,mut,strain,exp,00.A,00.C,00.G,00.T,01.A,01.C,...,47.G,47.T,48.A,48.C,48.G,48.T,49.A,49.C,49.G,49.T
name,,,,,,,,,,,,,,,,,,,,,
exp9_ars416_wtc1,ars416,WT,wtc1,exp9,0.006569,-0.009687,-0.008746,0.011863,0.017831,-0.011111,...,-0.077934,0.092319,0.000135,0.014144,-0.017737,0.003458,0.016456,-0.013796,0.026779,-0.029439
exp4_ars416_m1,ars416,Y486Q,m1,exp4,0.001342,-0.012931,-0.002719,0.014308,0.035252,-0.025139,...,-0.096519,0.116963,-0.015650,0.041240,-0.019375,-0.006215,0.038332,-0.013962,0.041598,-0.065968
exp7_ars416_m1,ars416,Y486Q,m1,exp7,0.004235,-0.000084,-0.006088,0.001938,-0.000695,-0.004777,...,-0.071717,0.140259,-0.014585,0.039943,-0.014460,-0.010899,0.047772,-0.033008,0.056662,-0.071426
exp9_ars416_m1c1,ars416,Y486Q,m1c1,exp9,-0.004067,0.001031,-0.002436,0.005473,0.002079,-0.000815,...,-0.112896,0.142464,-0.016034,0.045972,-0.015899,-0.014039,0.053347,-0.028282,0.045637,-0.070701
exp9_ars416_m1c2,ars416,Y486Q,m1c2,exp9,0.000685,-0.011873,0.009624,0.001564,0.005142,-0.008739,...,-0.088257,0.133616,-0.014247,0.033025,-0.010067,-0.008711,0.047386,-0.023742,0.041721,-0.065365


## Check files, esp. ordering of matrix values

In [8]:
# Load data
data_df_ER = pd.read_csv('data_ER.txt', delimiter='\t', index_col=0)
data_df_IM = pd.read_csv('data_IM.txt', delimiter='\t', index_col=0)

# Add 'lm' column and modify names accordingly
data_df_ER.insert(column='lm', value='ER', loc=0)
data_df_ER.index = [n+'_ER' for n in data_df_ER.index]
data_df_IM.insert(column='lm', value='IM', loc=0)
data_df_IM.index = [n+'_IM' for n in data_df_IM.index]

# Concatenate into a single dataframe and preview
data_df = pd.concat([data_df_IM, data_df_ER])
data_df.head()

,lm,ars,mut,strain,exp,00.A,00.C,00.G,00.T,01.A,...,47.G,47.T,48.A,48.C,48.G,48.T,49.A,49.C,49.G,49.T
exp6_ars317_m3_IM,IM,ars317,F485I,m3,exp6,-0.004045,0.005569,-0.014642,0.013118,-0.008004,...,-0.079839,0.111333,-0.029961,0.035882,-0.035030,0.029108,0.013203,-0.013798,0.004710,-0.004115
exp7_ars317_m2_IM,IM,ars317,F485I,m2,exp7,-0.016780,-0.004333,-0.000826,0.021939,-0.001462,...,-0.078331,0.100685,-0.028536,0.038137,-0.043542,0.033941,0.020821,-0.014027,0.002652,-0.009445
exp6_ars317_m21_IM,IM,ars317,FY-AA,m21,exp6,0.005840,0.003235,-0.033633,0.024558,-0.011041,...,-0.099109,0.133206,-0.040175,0.055051,-0.039498,0.024623,0.008800,-0.006307,0.009684,-0.012177
exp7_ars317_m21_IM,IM,ars317,FY-AA,m21,exp7,0.007108,-0.002196,-0.022928,0.018016,-0.016403,...,-0.094059,0.133311,-0.035379,0.047618,-0.031899,0.019660,0.014209,-0.019739,0.018216,-0.012686
exp5_ars317_m22_IM,IM,ars317,FY-IQ,m22,exp5,-0.012484,0.023423,-0.027210,0.016271,-0.021774,...,-0.067821,0.124411,-0.000793,0.016263,-0.064366,0.048896,0.010426,-0.000222,-0.016731,0.006528


In [9]:
# Save
df_file = 'data.txt'
data_df.to_csv(df_file, sep='\t')
print(f'Saved all matrices to {df_file}.')

Saved all matrices to data.txt.
